## Decision Trees Implementation
Uses Information Gain as the measurement to be split


In [308]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [309]:
data = datasets.load_iris()

In [310]:
data

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [311]:
names = data['target_names']

In [312]:
X = data['data']
X = np.array(X, dtype = int)
y = data['target']

In [313]:
data_ = pd.DataFrame(X)
data_.columns = data['feature_names']
data_['Output'] = Y

In [314]:
data['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [315]:
data_.isna().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
Output               0
dtype: int64

In [316]:
# Define Entropy and Information Gain

In [317]:
def entropy(col):
    
    counts = np.unique(col,return_counts=True)
    N = float(col.shape[0])
    
    ent = 0.0
    
    for ix in counts[1]:
        p  = ix/N
        ent += (-1.0*p*np.log2(p))
    
    return ent

In [318]:
def information_gain(x_data,key,val):
    left,right = divide_data(x_data,key,val)
    
    #% of total samples are on left and right
    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]
    
    #All examples come to one side!
    if left.shape[0] == 0 or right.shape[0] ==0:
        return -1000000 #Min Information Gain
    
    i_gain = entropy(x_data.Output) - (l*entropy(left.Output)+r*entropy(right.Output))
    return i_gain


In [319]:
def divide_data(x_data,key,val):
    #Work with Pandas Data Frames
    x_right = pd.DataFrame([],columns=x_data.columns)
    x_left = pd.DataFrame([],columns=x_data.columns)
    
    for ix in range(x_data.shape[0]):
        val_ = x_data[key].loc[ix]
        
        if val_ > val:
            x_right = x_right.append(x_data.loc[ix])
        else:
            x_left = x_left.append(x_data.loc[ix])
            
    return x_left,x_right


In [320]:
def find_count(X_train):                                   ###maintain the count of 0,1 and 2 for respectively
        count = []
        count.append(X_train[X_train['Output'] == 0].shape[0])
        count.append(X_train[X_train['Output'] == 1].shape[0])
        count.append(X_train[X_train['Output'] == 2].shape[0])
        return count

In [321]:
class DecisionTree:
    
    #Constructor
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.key = None
        self.val = None
        self.count = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
    
    def train(self,X_train,names):
        
        features = ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']
        info_gains = []
        
        for ix in features:
            i_gain = information_gain(X_train,ix,X_train[ix].mean())
            info_gains.append(i_gain)
            
        self.key = features[np.argmax(info_gains)]
        self.val = X_train[self.key].mean()
        print("Level " , self.depth)
        self.count = find_count(X_train)
        cnt = 0                            ##maintain to get if we have only 1 flower in this node
        for i in range(len(self.count)):
            if(self.count[i]):
                print("Count of " , names[i] , " = " , self.count[i])
                cnt += 1
        print("Current entropy = " , entropy(X_train.Output))
        if cnt != 1:
            print("Splitting on Tree Features ",self.key,"with information gain",np.argmax(info_gains))
        
        #Split Data
        data_left,data_right = divide_data(X_train,self.key,self.val)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
         
        #Truly a left node
        if cnt == 1:
            if X_train.Output.mean() >= 1.5:
                self.target = names[2]
            elif X_train.Output.mean() <= 0.5:
                self.target = names[0]
            else:
                self.target = names[1]
            print("Reached leaf Node")
            print()
            print()
            return
        
        #Stop earyly when depth >=max depth
        if(self.depth>=self.max_depth):
            if X_train.Output.mean() >= 1.5:
                self.target = names[2]
            elif X_train.Output.mean() <= 0.5:
                self.target = names[0]
            else:
                self.target = names[1]
            print("Max depth Reached")
            print()
            print()
            return
        
        print()
        print()
        
        #Recursive Case
        self.left = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left, names)
        
        self.right = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right, names)
        
        #You can set the target at every node
        if X_train.Output.mean() >= 1.5:
            self.target = names[2]
        elif X_train.Output.mean() <= 0.5:
            self.target = names[0]
        else:
            self.target = names[1]
        return    

In [322]:
dt = DecisionTree()     #making object

In [323]:
dt.train(data_, names)      ##training or building tree

Level  0
Count of  setosa  =  50
Count of  versicolor  =  50
Count of  virginica  =  50
Current entropy =  1.584962500721156
Splitting on Tree Features  petal width (cm) with information gain 3


Level  1
Count of  setosa  =  50
Current entropy =  0.0
Reached leaf Node


Level  1
Count of  versicolor  =  50
Count of  virginica  =  50
Current entropy =  1.0
Splitting on Tree Features  petal length (cm) with information gain 2


Level  2
Count of  versicolor  =  48
Count of  virginica  =  6
Current entropy =  0.5032583347756457
Splitting on Tree Features  petal width (cm) with information gain 3


Level  3
Count of  versicolor  =  48
Count of  virginica  =  5
Current entropy =  0.45079138835466503
Splitting on Tree Features  petal length (cm) with information gain 2


Level  4
Count of  versicolor  =  11
Current entropy =  0.0
Reached leaf Node


Level  4
Count of  versicolor  =  37
Count of  virginica  =  5
Current entropy =  0.5266170655714281
Splitting on Tree Features  sepal length (